In [2]:

import scipy as sp
import scipy.linalg as la
# import scipy.random as rand
import scipy.sparse as spr

def kr(a,b): return sp.kron(a,b)
def kr4(a,b,c,d): return kr(kr(kr(a,b),c),d)
def dot(a,b): return sp.dot(a,b)
def dot4(a,b,c,d): return dot(dot(dot(a,b),c),d)
def krx(*mats):
        res = 1.
        for i,M in enumerate(mats):
                res = kr(res,M)
        return res

def dotx(*mats):
        res = 1.
        for i,M in enumerate(mats):
                res = dot(res,M)
        return res



In [3]:
Hd = sp.array([[1+0j,1],[1,-1]])/sp.sqrt(2.)
X = sp.array([[0+0j,1],[1, 0]])
Y = sp.array([[0,-1j],[1j,0]])
Z = sp.array([[1+0j,0],[0,-1]])
I = sp.eye(2,dtype=complex)
cnot = sp.zeros((4,4),dtype=complex)
cnot[0,0]=cnot[1,1]=cnot[2,3]=cnot[3,2] = 1.

prj0 = sp.zeros((2,2),dtype=complex); prj0[0,0]=1.
prj1 = sp.zeros((2,2),dtype=complex); prj1[1,1]=1.


Rx = lambda th: sp.eye(2)*sp.cos(th/2) - 1j*X*sp.sin(th/2)
Ry = lambda th: sp.eye(2)*sp.cos(th/2) - 1j*Y*sp.sin(th/2)
Rz = lambda th: sp.eye(2)*sp.cos(th/2) - 1j*Z*sp.sin(th/2)

# Rx = lambda th: sp.eye(2)*sp.cos(th) + 1j*X*sp.sin(th)
# Ry = lambda th: sp.eye(2)*sp.cos(th) + 1j*Y*sp.sin(th)
# Rz = lambda th: sp.eye(2)*sp.cos(th) + 1j*Z*sp.sin(th)


In [4]:
print Rx(sp.pi)

[[  6.12323400e-17+0.j   0.00000000e+00-1.j]
 [  0.00000000e+00-1.j   6.12323400e-17+0.j]]


In [5]:
def crCNOThilb(i1,i2,nq):
    
    # cnot = kr(prj0,I) + kr(prj1,X) = A + B
    
    A = 1
    B = 1
    for ind in range(nq):
        if ind==i1:
            A = kr(A,prj0)
            B = kr(B,prj1)
        elif ind==i2:
            A = kr(A,I)
            B = kr(B,X)
        else:
            A = kr(A,I)
            B = kr(B,I)
    
    return A + B


def getHilbGate(gate,qubind,nq):
    
    G = 1
    for i in range(nq):
        
        if i==qubind:
            G = kr(G,gate)
        else:
            G = kr(G,I)
        
    return G


In [6]:
print crCNOThilb(0,1,2)
print crCNOThilb(1,0,2)



[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]]
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]]


In [7]:

'''
Create random circuit
nq: number of qubits
cnPercyc: number of cnots per cycle

(implicitly, initial state is |0>)
output: (ham, qasm)
'''
def crRandCirc(nq, cnPerCyc, totCyc, qasmlbl=""):
    
    
    # Input check
    if 2*cnPerCyc > nq:
        raise Exception("# CNOT's per cycle must be less than half of # qubits.")
    
    # Start hamiltonian
    hilbsz = 2**nq
    H = sp.eye(hilbsz)
    
    # Start up the file
    nl = "\n"
    qasm = str(nq)+nl
    qasm += "# Rand circ: nq="+str(nq)+" cnPerCyc="+str(cnPerCyc)+" totCyc="+str(totCyc)+"  "+qasmlbl+nl
    
    # For convenience, store lambda functions
    rotOps = [Rx, Ry, Rz]; rotStrs = ['Rx','Ry','Rz']
#     rotOps = [Rx, Rz]; rotStrs = ['Rx','Rz']
#     rotOps = [Rx]; rotStrs = ['Rx']

    
    for cyc in range(totCyc):
        
        # Remaining qubits to place
        qubRemain = sp.arange(nq).tolist()
        
        # First place cnots
        for icn in range(cnPerCyc):
            
            a = -1
            while not (a in qubRemain):
                a = sp.random.randint(nq)
            qubRemain.remove(a)
            
            b = -1
            while not (b in qubRemain):
                b = sp.random.randint(nq)
            qubRemain.remove(b)
            
            # Add to QASM file
            qasm += "CNOT " + str(a) + " " + str(b) + nl
            
#             # Add to Ham
#             H = sp.dot( crCNOThilb(a,b,nq) ,H)
            
        # Place rotations on the rest
        for ind in qubRemain:
            
            direc = sp.random.randint( len(rotOps) )
            th = sp.around( 0.1*sp.random.rand() , 4 )
            
            # Add to QASM file
            qasm += rotStrs[direc] + " " + str(th) + " " + str(ind) + nl
            
#            # Add to Ham
#             gate = getHilbGate(  rotOps[direc](th)  ,ind,nq)  
#             H = sp.dot( gate, H )
    
#     return (H,qasm)
    return qasm
    

In [8]:

cmt="""
nq = 6
cnPerCyc = 2
totCyc = 10

# (H,qasm) = crRandCirc(nq, cnPerCyc, totCyc, "Rx-only")
qasm = crRandCirc(nq, cnPerCyc, totCyc, "RxRzRz")


print qasm
f = open("xxxxrand-nq"+str(nq)+"-cn"+str(cnPerCyc)+"-d"+str(totCyc)+"_rxyz.qasm",'w')
f.write(qasm)
f.close()
"""


In [23]:
def printvec(psi):
    print "State vector:"
    for cf in psi:
        print cf
    print ""

# Read in the qasm, output a Hamiltonian
def qasmToHam(qasmfname):
    
    f = open(qasmfname,'r')
    l = f.readline()
    
    nq = int(l)
    print "numqub="+str(nq)
    hilbsz = 2**nq
    
    H = sp.eye(hilbsz)
    
    psi = sp.zeros(hilbsz,dtype=complex); psi[0]=1.0
    printvec(psi)
    
    l = f.readline()
    ind = 1
    while( l ):
        if l[0]!='#':
            
            gstr = l.split()
            g = gstr[0]
            if g=="CNOT":
                i1 = int(gstr[1]); i2=int(gstr[2])
                gate = crCNOThilb( i1,i2,nq)
                H = sp.dot(gate,H)
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="Rx":
                th = float(gstr[1])
                gate = getHilbGate(  Rx(th)  , int(gstr[2]) ,nq)  
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="Ry":
                th = float(gstr[1])
                gate = getHilbGate(  Ry(th)  , int(gstr[2]) ,nq)  
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="Rz":
                th = float(gstr[1])
                gate = getHilbGate(  Rz(th)  , int(gstr[2]) ,nq)  
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="H":
                thequb = int(gstr[1])
                gate = getHilbGate( Hd , thequb , nq )
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="X":
                thequb = int(gstr[1])
                thequb = getHilbGate( X , thequb , nq )
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
                
            if g=="Y":
                thequb = int(gstr[1])
                thequb = getHilbGate( Y , thequb , nq )
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                printvec(psi)
                
            if g=="Z":
                thequb = int(gstr[1])
                thequb = getHilbGate( Z , thequb , nq )
                H = sp.dot( gate, H )
                psi = sp.dot( gate , psi )
                print ind
                ind = ind + 1
                printvec(psi)
            
            
        l = f.readline()
            
    
    return [H,nq,psi]
    

In [27]:

pr = "tests-sept2017/tests"

strQasmFiles= [ pr+"rand-nq6-cn2-d10_rx.qasm",
                pr+"rand-nq6-cn2-d10_rxyz.qasm",
               pr+"rand-nq6-cn2-d10_rxz.qasm",
               pr+"test_JW.qasm"
            ]

[H,nq,psi] = qasmToHam("tests/rand-nq6-cn2-d10_rxz.qasm")


hilbsz = 2**nq

# Read and measure
measureXI = [X,I,X,I,X,I,X,I,X,I,X,I,X,I]
measureYI = [Y,I,Y,I,Y,I,Y,I,Y,I,Y,I,Y,I]
measureZI = [Z,I,Z,I,Z,I,Z,I,Z,I,Z,I,Z,I]

measureOpX = 1
measureOpY = 1
measureOpZ = 1
for i in range(nq):

    measureOpX = sp.kron(measureOpX,measureXI[i])
    measureOpY = sp.kron(measureOpY,measureYI[i])
    measureOpZ = sp.kron(measureOpZ,measureZI[i])

measPsiX = sp.dot( measureOpX , psi )
measPsiY = sp.dot( measureOpY , psi )
measPsiZ = sp.dot( measureOpZ , psi )
    
resX = sp.dot( sp.conj(psi) , measPsiX )
resY = sp.dot( sp.conj(psi) , measPsiY )
resZ = sp.dot( sp.conj(psi) , measPsiZ )
print "Measure with XIXI..."
print resX
print "Measure with YIYI..."
print resY
print "Measure with ZIZI..."
print resZ
    
print ""


numqub=6
State vector:
(1+0j)
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j

1
State vector:
(1+0j)
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j

2
State vector:
(1+0j)
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j

3
State vector:
(0.999994718755+0j)
0j
0j
0j
-0.00324999427865j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j

4
State vector:
(0.999446966661+0j)
0j
-0.0330937814386j
0j
-0.00324821407808j
0j
(-0.00010755516836+0j)
0j
0j
0